In [57]:
import tensorflow as tf
import keras 
from keras.applications import VGG16, DenseNet121, DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers

In [58]:
conv_base1 = VGG16(weights='imagenet', include_top = False, input_shape=(300,300,3))
conv_base2 = DenseNet121(weights='imagenet', include_top = False, input_shape=(300,300,3))
conv_base3 = DenseNet201(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [59]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [4]:
conv_base1.predict(X[0].reshape(1,300,300,3)).shape, conv_base2.predict(X[0].reshape(1,300,300,3)).shape, conv_base3.predict(X[0].reshape(1,300,300,3)).shape

((1, 9, 9, 512), (1, 9, 9, 1024), (1, 9, 9, 1920))

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [5]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [60]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [15]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


In [16]:
conv1_samples = np.zeros(shape=(nos,9,9,512))
conv1_labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base1.predict(samples_batch)
    print(f.shape)
    conv1_samples[i*bs:(i+1)*bs] = f
    conv1_labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)


In [17]:
conv1_samples = conv1_samples.reshape(nos,9*9*512)

In [18]:
conv2_samples = np.zeros(shape=(nos,9,9,1024))
conv2_labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base2.predict(samples_batch)
    print(f.shape)
    conv2_samples[i*bs:(i+1)*bs] = f
    conv2_labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)


In [19]:
conv2_samples = conv2_samples.reshape(nos,9*9*1024)

In [20]:
conv3_samples = np.zeros(shape=(nos,9,9,1920))
conv3_labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base3.predict(samples_batch)
    print(f.shape)
    conv3_samples[i*bs:(i+1)*bs] = f
    conv3_labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)


In [21]:
conv3_samples = conv3_samples.reshape(nos,9*9*1920)

In [16]:
conv1_samples.shape, conv1_labels.shape, conv2_samples.shape, conv2_labels.shape, conv3_samples.shape, conv3_labels.shape

((472, 41472), (472,), (472, 82944), (472,), (472, 155520), (472,))

In [62]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

In [64]:
accuracies = []
f = 1

for train_index, test_index in skf.split(X, y):
        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]
      
        X_train1 = conv_base1.predict(X_train)
        X_train1 = X_train1.reshape(X_train.shape[0],9*9*512)
        X_test1 = conv_base1.predict(X_test)
        X_test1 = X_test1.reshape(X_test.shape[0],9*9*512)
        
        X_train2 = conv_base2.predict(X_train)
        X_train2 = X_train2.reshape(X_train.shape[0],9*9*1024)
        X_test2 = conv_base2.predict(X_test)
        X_test2 = X_test2.reshape(X_test.shape[0],9*9*1024)
        
        X_train3 = conv_base3.predict(X_train)
        X_train3 = X_train3.reshape(X_train.shape[0],9*9*1920)
        X_test3 = conv_base3.predict(X_test)
        X_test3 = X_test3.reshape(X_test.shape[0],9*9*1920)

        model1 = models.Sequential()
        model1.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
        model1.add(layers.Dropout(0.5))
        model1.add(layers.Dense(1, activation = 'sigmoid'))
        model1.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        
        model2 = models.Sequential()
        model2.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1024))
        model2.add(layers.Dropout(0.5))
        model2.add(layers.Dense(1, activation = 'sigmoid'))
        model2.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        
        model3 = models.Sequential()
        model3.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
        model3.add(layers.Dropout(0.5))
        model3.add(layers.Dense(1, activation = 'sigmoid'))
        model3.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        
        
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

        print('Fold {}'.format(f))
        
        history1 = model1.fit(X_train1, y_train, epochs = 100, batch_size = 20, validation_data = (X_test1, y_test), shuffle = True, callbacks = my_callbacks)
        history2 = model2.fit(X_train2, y_train, epochs = 100, batch_size = 20, validation_data = (X_test2, y_test), shuffle = True, callbacks = my_callbacks)
        history3 = model3.fit(X_train3, y_train, epochs = 100, batch_size = 20, validation_data = (X_test3, y_test), shuffle = True, callbacks = my_callbacks)


        test_preds1 = model1.predict(X_test1, batch_size = 20)
        test_predclass1 = np.array([1.0 if i>0.5 else 0.0 for i in test_preds1])

        test_preds2 = model2.predict(X_test2, batch_size = 20)
        test_predclass2 = np.array([1.0 if i>0.5 else 0.0 for i in test_preds2])

        test_preds3 = model3.predict(X_test3, batch_size = 20)
        test_predclass3 = np.array([1.0 if i>0.5 else 0.0 for i in test_preds3])

        kfinal_preds = np.empty(test_predclass3.shape[0])
        for i in range(test_predclass3.shape[0]):
          predsum = test_predclass1[i] + test_predclass2[i] + test_predclass3[i] 
          kfinal_preds[i] = 0.0 if predsum in [0.0,1.0] else 1.0


        accuracies.append([confusion_matrix(y_test,kfinalpreds)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,kfinalpreds)[1][1]/(y_test == 1.0).sum()])
        f+=1

ValueError: ignored

In [91]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.7895114942528736 0.8475574712643679


Training on whole Training Dataset

In [23]:
model_conv1 = models.Sequential()
model_conv1.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
model_conv1.add(layers.Dropout(0.5))
model_conv1.add(layers.Dense(1, activation = 'sigmoid'))
model_conv1.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])

model_conv2 = models.Sequential()
model_conv2.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1024))
model_conv2.add(layers.Dropout(0.5))
model_conv2.add(layers.Dense(1, activation = 'sigmoid'))
model_conv2.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])

model_conv3 = models.Sequential()
model_conv3.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
model_conv3.add(layers.Dropout(0.5))
model_conv3.add(layers.Dense(1, activation = 'sigmoid'))
model_conv3.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])

my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

In [24]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [25]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)
datagentest.fit(xtest)
testnos = ytest.shape[0]


numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


In [28]:
conv1_test_samples = np.zeros(shape=(testnos,9,9,512))
conv1_test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base1.predict(samples_batch)
    print(g.shape)
    conv1_test_samples[::] = g
    conv1_test_labels[:] = labels_batch
    break

(552, 9, 9, 512)


In [30]:
conv1_test_samples = conv1_test_samples.reshape(testnos,9*9*512)

In [32]:
conv2_test_samples = np.zeros(shape=(testnos,9,9,1024))
conv2_test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base2.predict(samples_batch)
    print(g.shape)
    conv2_test_samples[::] = g
    conv2_test_labels[:] = labels_batch
    break

(552, 9, 9, 1024)


In [33]:
conv2_test_samples = conv2_test_samples.reshape(testnos,9*9*1024)

In [35]:
conv3_test_samples = np.zeros(shape=(testnos,9,9,1920))
conv3_test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base3.predict(samples_batch)
    print(g.shape)
    conv3_test_samples[::] = g
    conv3_test_labels[:] = labels_batch
    break

(552, 9, 9, 1920)


In [36]:
conv3_test_samples = conv3_test_samples.reshape(testnos,9*9*1920)

In [101]:
# To save memory
del xtest
del ytest

In [37]:
conv1_historytest = model_conv1.fit(conv1_samples, conv1_labels, epochs = 100, batch_size = 20, validation_data = (conv1_test_samples, conv1_test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 1s 2ms/step - loss: 2.9841 - accuracy: 0.6335 - val_loss: 0.8218 - val_accuracy: 0.7627
Epoch 2/100
472/472 [==============================] - 1s 1ms/step - loss: 0.5000 - accuracy: 0.8453 - val_loss: 0.5651 - val_accuracy: 0.7717
Epoch 3/100
472/472 [==============================] - 1s 1ms/step - loss: 0.2558 - accuracy: 0.8941 - val_loss: 0.5697 - val_accuracy: 0.7935
Epoch 4/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1679 - accuracy: 0.9322 - val_loss: 0.7086 - val_accuracy: 0.7373
Epoch 5/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1735 - accuracy: 0.9470 - val_loss: 0.3871 - val_accuracy: 0.8641
Epoch 6/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1363 - accuracy: 0.9492 - val_loss: 0.7219 - val_accuracy: 0.7464
Epoch 7/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1206 - accuracy: 0.

In [38]:
conv2_historytest = model_conv2.fit(conv2_samples, conv2_labels, epochs = 100, batch_size = 20, validation_data = (conv2_test_samples, conv2_test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 1s 3ms/step - loss: 7.6073 - accuracy: 0.6568 - val_loss: 2.3205 - val_accuracy: 0.8062
Epoch 2/100
472/472 [==============================] - 1s 2ms/step - loss: 2.7081 - accuracy: 0.7945 - val_loss: 1.3893 - val_accuracy: 0.8442
Epoch 3/100
472/472 [==============================] - 1s 2ms/step - loss: 0.7197 - accuracy: 0.8623 - val_loss: 0.7213 - val_accuracy: 0.8351
Epoch 4/100
472/472 [==============================] - 1s 2ms/step - loss: 0.2725 - accuracy: 0.9131 - val_loss: 1.2570 - val_accuracy: 0.6957
Epoch 5/100
472/472 [==============================] - 1s 2ms/step - loss: 0.2322 - accuracy: 0.9258 - val_loss: 0.6513 - val_accuracy: 0.7717
Epoch 6/100
472/472 [==============================] - 1s 2ms/step - loss: 0.1597 - accuracy: 0.9322 - val_loss: 0.6958 - val_accuracy: 0.7536
Epoch 7/100
472/472 [==============================] - 1s 2ms/step - loss: 0.1428 - accuracy: 0.

In [39]:
conv3_historytest = model_conv3.fit(conv3_samples, conv3_labels, epochs = 100, batch_size = 20, validation_data = (conv3_test_samples, conv3_test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 2s 5ms/step - loss: 7.9190 - accuracy: 0.6483 - val_loss: 2.0795 - val_accuracy: 0.8714
Epoch 2/100
472/472 [==============================] - 2s 4ms/step - loss: 1.9778 - accuracy: 0.8093 - val_loss: 1.3705 - val_accuracy: 0.7409
Epoch 3/100
472/472 [==============================] - 2s 4ms/step - loss: 0.5379 - accuracy: 0.8898 - val_loss: 1.5802 - val_accuracy: 0.6649
Epoch 4/100
472/472 [==============================] - 2s 4ms/step - loss: 0.5766 - accuracy: 0.8919 - val_loss: 0.7267 - val_accuracy: 0.8551
Epoch 5/100
472/472 [==============================] - 2s 4ms/step - loss: 0.2672 - accuracy: 0.9068 - val_loss: 0.7585 - val_accuracy: 0.7264
Epoch 6/100
472/472 [==============================] - 2s 4ms/step - loss: 0.1883 - accuracy: 0.9407 - val_loss: 0.5178 - val_accuracy: 0.8551
Epoch 7/100
472/472 [==============================] - 2s 4ms/step - loss: 0.1273 - accuracy: 0.

In [40]:
conv1_test_preds = model_conv1.predict(conv1_test_samples, batch_size = 20)
conv1_test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in conv1_test_preds])

In [41]:
conv2_test_preds = model_conv2.predict(conv2_test_samples, batch_size = 20)
conv2_test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in conv2_test_preds])

In [42]:
conv3_test_preds = model_conv3.predict(conv3_test_samples, batch_size = 20)
conv3_test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in conv3_test_preds])

In [55]:
# Voting

final_preds = np.empty(conv1_test_predclass.shape[0])
for i in range(conv1_test_predclass.shape[0]):
  predsum = conv1_test_predclass[i] + conv2_test_predclass[i] + conv3_test_predclass[i] 
  final_preds[i] = 0.0 if predsum in [0.0,1.0] else 1.0

In [56]:
confusion_matrix(conv1_test_labels,final_preds)[0][0]/(conv1_test_labels == 0.0).sum(), confusion_matrix(conv1_test_labels,final_preds)[1][1]/(conv1_test_labels == 1.0).sum()

(0.6610169491525424, 0.922920892494929)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/